In [12]:
import pywt
import os
import pandas as pd
import librosa
import librosa.display
import glob 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from fitter import Fitter, get_common_distributions, get_distributions
import random

from sklearn.preprocessing import StandardScaler
import sklearn

from utils import plot_projections

from scipy.stats import kurtosis, skew

#parselmouth
import parselmouth 
import statistics
from parselmouth.praat import call
from scipy.stats.mstats import zscore
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

%matplotlib widget

In [7]:
laser_dir = '/home/hashim/PHD/audio_data/AllAudioSamples/Laser/'
laser_files = list(glob.glob(os.path.join(laser_dir, '*.wav')))
laser_files.sort()
print(laser_files)
print(len(laser_files))

['/home/hashim/PHD/audio_data/AllAudioSamples/Laser/laser001 (2).wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser/laser001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser/laser002 (2).wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser/laser002.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser/laser003 (2).wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser/laser003.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser/laser004 (2).wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser/laser004.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser/laser005 (2).wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser/laser005.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser/laser006 (2).wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser/laser006.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser/laser007 (2).wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser/laser007.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/La

In [8]:
orig_dir = '/home/hashim/PHD/audio_data/AllAudioSamples/Original/'
orig_files = list(glob.glob(os.path.join(orig_dir, '*wav')))
orig_files.sort()
print(orig_files)
print(len(orig_files))

['/home/hashim/PHD/audio_data/AllAudioSamples/Original/original001 (2).wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original/original001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original/original002 (2).wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original/original002.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original/original003 (2).wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original/original003.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original/original004 (2).wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original/original004.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original/original005 (2).wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original/original005.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original/original006 (2).wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original/original006.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original/original007 (2).wav', '/home/hashim/PHD/audio_data/AllAud

In [9]:
# generate a random integer between 0 and minimum of length of original and laser files. 

audio_idx = random.randint(0, min(len(laser_files), len(orig_files)))
audio_idx

56

In [10]:
laser_audio, sr = librosa.load(laser_files[audio_idx], res_type='kaiser_fast')
auth_audio, sr = librosa.load(orig_files[audio_idx], res_type='kaiser_fast')

In [13]:
# This function measures formants using Formant Position formula
def measureFormants(sound, wave_file, f0min,f0max):
    sound = parselmouth.Sound(sound) # read the sound
    pitch = call(sound, "To Pitch (cc)", 0, f0min, 15, 'no', 0.03, 0.45, 0.01, 0.35, 0.14, f0max)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    
    formants = call(sound, "To Formant (burg)", 0.0025, 5, 5000, 0.025, 50)
    numPoints = call(pointProcess, "Get number of points")

    f1_list = []
    f2_list = []
    f3_list = []
    f4_list = []
    
    # Measure formants only at glottal pulses
    for point in range(0, numPoints):
        point += 1
        t = call(pointProcess, "Get time from index", point)
        f1 = call(formants, "Get value at time", 1, t, 'Hertz', 'Linear')
        f2 = call(formants, "Get value at time", 2, t, 'Hertz', 'Linear')
        f3 = call(formants, "Get value at time", 3, t, 'Hertz', 'Linear')
        f4 = call(formants, "Get value at time", 4, t, 'Hertz', 'Linear')
        f1_list.append(f1)
        f2_list.append(f2)
        f3_list.append(f3)
        f4_list.append(f4)
    
    f1_list = [f1 for f1 in f1_list if str(f1) != 'nan']
    f2_list = [f2 for f2 in f2_list if str(f2) != 'nan']
    f3_list = [f3 for f3 in f3_list if str(f3) != 'nan']
    f4_list = [f4 for f4 in f4_list if str(f4) != 'nan']
    
    # calculate mean formants across pulses
    f1_mean = statistics.mean(f1_list)
    f2_mean = statistics.mean(f2_list)
    f3_mean = statistics.mean(f3_list)
    f4_mean = statistics.mean(f4_list)
    
    # calculate median formants across pulses, this is what is used in all subsequent calcualtions
    # you can use mean if you want, just edit the code in the boxes below to replace median with mean
    f1_median = statistics.median(f1_list)
    f2_median = statistics.median(f2_list)
    f3_median = statistics.median(f3_list)
    f4_median = statistics.median(f4_list)
    
    return f1_mean, f2_mean, f3_mean, f4_mean, f1_median, f2_median, f3_median, f4_median


In [15]:
sound_laser = parselmouth.Sound(laser_files[audio_idx])
sound_auth = parselmouth.Sound(orig_files[audio_idx])

In [17]:
(f1_mean_laser, f2_mean_laser, f3_mean_laser, f4_mean_laser, f1_median_laser, f2_median_laser, f3_median_laser, f4_median_laser) = measureFormants(
        sound, laser_files[audio_idx], 75, 300)

In [18]:
f1_mean_laser

556.629743313827